In [3]:
import pandas as pd
import streamlit as st 
import plotly.express as px

In [88]:

def ler_excel(excel):

    df = pd.read_excel(excel)
    if excel.find("2025") > 0:
        if excel.find("_TEC") > 0:
            df[['Curso','Campus','Turno']]  = df['Cargo'].str.split('-',expand=True)
        else:
            df[['Curso','Modalidade','Campus','Turno', 'FormaIngresso']]  = df['Cargo'].str.split('-',expand=True)
    else:
        df[['Curso','Campus','Turno']]  = df['Cargo'].str.split('-',expand=True)

    df  = df[['Curso','Campus','Turno',"Nivel","Inscritos","Pagos", "Isenções deferidas","Inscrições homologadas"]]
    df.rename({"Isenções deferidas": 'Deferidas', "Inscrições homologadas": 'Homologadas'}, axis=1, inplace=True) 
    df['Campus'] = df['Campus'].apply(lambda r: r.replace("Campus","").replace("campus","").strip().upper() )
    df['Curso'] = df['Curso'].apply(lambda r: r
                                    .replace("Técnico Integrado em","")
                                    .replace("Técnico Integrado","")
                                    .replace("Técnico Subsequente em","").strip().upper())

    return df 

def diferenca(ano1,ano2):

    dfMed_ano1 = ler_excel(ano1)
    dfMed_ano2 = ler_excel(ano2)


    dff = pd.DataFrame()

    dff['Curso'] = dfMed_ano1['Curso']
    dff['Campus'] = dfMed_ano1['Campus']
    dff['Nivel'] = dfMed_ano1['Nivel']
    dff['ano1_Inscritos'] = dfMed_ano1['Inscritos']
    dff['ano2_Inscritos'] = dfMed_ano2['Inscritos']
    dff['ano1_Homologadas'] = dfMed_ano1['Homologadas']
    dff['ano2_Homologadas'] = dfMed_ano2['Homologadas']
    
    dff['diff_Inscritos'] = dfMed_ano2['Inscritos'] - dfMed_ano1['Inscritos']
    dff['diff_Homologadas'] = dfMed_ano2['Homologadas'] - dfMed_ano1['Homologadas']

    return dff


def diff(df1, df2, tipo="Curso"):
    
    df11 = df1.groupby(tipo)[["Inscritos","Pagos", "Deferidas","Homologadas"]].sum().reset_index().sort_values(by='Inscritos', ascending=False)
    df11.set_index(tipo, inplace=True)

    df22 = df2.groupby(tipo)[["Inscritos","Pagos", "Deferidas","Homologadas"]].sum().reset_index().sort_values(by='Inscritos', ascending=False)
    df22.set_index(tipo, inplace=True)

    return (df22 - df11).reset_index().sort_values("Inscritos")

In [106]:
def process_file_for_grad(file_path):
    """
    Function to process files for Graduação (Superior).
    Renames 'Tipo de Vaga' to 'Forma de Ingresso' for Graduação level.
    """
    df = pd.read_excel(file_path)
    
    # Splitting the "Cargo" column into 'Curso', 'Modalidade', 'Campus', 'Turno', 'Tipo de Vaga'
    cargo_split = df['Cargo'].str.split(' - ', expand=True)
    
    if cargo_split.shape[1] == 3:
        df['Curso'] = cargo_split[0]
        df['Campus'] = cargo_split[1]
        df['Turno'] = cargo_split[2]
        df['Modalidade'] = 'Superior'
        df['FormaIngresso'] = df['Tipo de Vaga']
    else: 
        df['Curso'] = cargo_split[0]
        df['Modalidade'] = cargo_split[1]
        df['Campus'] = cargo_split[2]
        df['Turno'] = cargo_split[3]
        df['FormaIngresso'] = cargo_split[4]
    

    df.rename({"Isenções deferidas": 'Deferidas', "Inscrições homologadas": 'Homologadas'}, axis=1, inplace=True) 
    df['Campus'] = df['Campus'].apply(lambda r: r.replace("Campus","").replace("campus","").strip().upper() )
    df['Curso'] = df['Curso'].apply(lambda r: r
                                    .replace("Tecnologia em","").strip().upper())


    # Selecting and reordering the desired columns
    final_df = df[['Curso', 'Modalidade', 'Campus', 'Turno', 'Nivel', 'Inscritos', 'Pagos', 'Deferidas', 'Homologadas', 'FormaIngresso']]
    
    return final_df


def process_file_for_tec(file_path):
    """
    Function to process files for Técnico level.
    Keeps the column name as 'Tipo de Vaga'.
    """
    df = pd.read_excel(file_path)
    
    # Splitting the "Cargo" column into 'Curso', 'Modalidade', 'Campus', 'Turno', 'Tipo de Vaga'
    cargo_split = df['Cargo'].str.split(' - ', expand=True)
    
    df['Curso'] = cargo_split[0]
    df['Campus'] = cargo_split[1]
    df['Turno'] = cargo_split[2]
    df['Modalidade'] = df['Tipo de Vaga'] if 'Tipo de Vaga' in df.columns else 'Curso Técnico Integrado'
    df['FormaIngresso'] = 'Processo Seletivo'
        
    
    df.rename({"Isenções deferidas": 'Deferidas', "Inscrições homologadas": 'Homologadas'}, axis=1, inplace=True) 
    df['Campus'] = df['Campus'].apply(lambda r: r.replace("Campus","").replace("campus","").strip().upper() )
    df['Curso'] = df['Curso'].apply(lambda r: r
                                    .replace("Técnico Integrado em","")
                                    .replace("Técnico Integrado","")
                                    .replace("Técnico Subsequente em","").strip().upper())

    # Selecting and reordering the desired columns
    final_df = df[['Curso', 'Modalidade', 'Campus', 'Turno', 'Nivel', 'Inscritos', 'Pagos', 
                   'Deferidas', 'Homologadas', 'FormaIngresso']]
    
    return final_df


In [19]:
df_tec_2024_1 = ler_excel("dados/2024-1_GestaoResultado_ResumoInscricoes_TEC.xlsx")
df_gra_2024_1 = ler_excel("dados/2024-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx")

df_tec_2025_1 = ler_excel("dados/2025-1_GestaoResultado_ResumoInscricoes_TEC.xlsx")
df_gra_2025_1 = ler_excel("dados/2025-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx")

In [23]:
df_tec_2024_1[df_tec_2024_1['Campus']=="PONTE NOVA"].head()

,Curso,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas
4,ADMINISTRAÇÃO,PONTE NOVA,Turno Integral,Nível Fundamental,339,282,28,310
30,INFORMÁTICA,PONTE NOVA,Turno Integral,Nível Fundamental,291,241,26,267


In [24]:
df_tec_2025_1[df_tec_2025_1['Campus']=="PONTE NOVA"].head()

,Curso,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas
37,ADMINISTRAÇÃO,PONTE NOVA,Turnos Manhã e Tarde,médio,58,24,0,24
38,INFORMÁTICA,PONTE NOVA,Turnos Manhã e Tarde,médio,73,24,0,24


In [21]:
df_gra_2025_1.head()

,Curso,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas
0,ENGENHARIA MECÂNICA,ARCOS,Turno Noite,Superior,15,6,0,6
1,ADMINISTRAÇÃO,BAMBUÍ,Turno Noturno,Superior,6,1,0,1
2,AGRONOMIA,SÃO JOÃO EVANGELISTA,Turno manhã e tarde,Superior,2,0,0,0
3,CIÊNCIAS BIOLÓGICAS,BAMBUÍ,Turno Noturno,Superior,0,0,0,0
4,CIÊNCIAS BIOLÓGICAS,BAMBUÍ,Turno Noturno,Superior,1,0,0,0


In [29]:
d = diferenca("dados/2024-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx",
              "dados/2025-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx")

d[d['Campus'] == 'PONTE NOVA']

,Curso,Campus,Nivel,ano1_Inscritos,ano2_Inscritos,ano1_Homologadas,ano2_Homologadas,diff_Inscritos,diff_Homologadas
68,TECNOLOGIA EM PROCESSOS GERENCIAIS,PONTE NOVA,Nível Médio,23,8,12,3,-15.0,-9.0
69,TECNOLOGIA EM PROCESSOS GERENCIAIS,PONTE NOVA,Nível Médio,35,5,19,2,-30.0,-17.0


In [89]:
diff(df_tec_2024_1, df_tec_2025_1, tipo=["Campus","Curso"])

,Campus,Curso,Inscritos,Pagos,Deferidas,Homologadas
49,SANTA LUZIA,EDIFICAÇÕES,-809.0,-634.0,-105.0,-739.0
11,BETIM,QUÍMICA,-797.0,-667.0,-73.0,-740.0
9,BETIM,AUTOMAÇÃO INDUSTRIAL,-734.0,-612.0,-72.0,-684.0
45,RIBEIRÃO DAS NEVES,INFORMÁTICA,-491.0,-366.0,-81.0,-447.0
10,BETIM,MECÂNICA,-432.0,-350.0,-33.0,-383.0
48,SABARÁ,INFORMÁTICA,-430.0,-358.0,-46.0,-404.0
22,GOVERNADOR VALADARES,SEGURANÇA DO TRABALHO,-420.0,-244.0,-73.0,-317.0
46,SABARÁ,ADMINISTRAÇÃO,-390.0,-315.0,-56.0,-371.0
26,IPATINGA,AUTOMAÇÃO INDUSTRIAL,-385.0,-313.0,-30.0,-343.0
23,IBIRITÉ,AUTOMAÇÃO INDUSTRIAL,-375.0,-328.0,-30.0,-358.0


In [98]:
process_file_for_grad("dados/2025-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx")

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,FormaIngresso
0,ENGENHARIA MECÂNICA,Bacharelado,ARCOS,Turno Noite,Superior,15,6,0,6,Redação Online
1,ADMINISTRAÇÃO,Bacharelado,BAMBUÍ,Turno Noturno,Superior,6,1,0,1,Notas do Enem
2,AGRONOMIA,Bacharelado,SÃO JOÃO EVANGELISTA,Turno manhã e tarde,Superior,2,0,0,0,Redação Online
3,CIÊNCIAS BIOLÓGICAS,Licenciatura,BAMBUÍ,Turno Noturno,Superior,0,0,0,0,Notas do Enem
4,CIÊNCIAS BIOLÓGICAS,Licenciatura,BAMBUÍ,Turno Noturno,Superior,1,0,0,0,Redação Online
...,...,...,...,...,...,...,...,...,...,...
83,MATEMÁTICA,Licenciatura,SÃO JOÃO EVANGELISTA,Turno Noturno,Superior,1,0,0,0,Notas do Enem
84,MATEMÁTICA,Licenciatura,SÃO JOÃO EVANGELISTA,Turno Noturno,Superior,0,0,0,0,Redação Online
85,SISTEMAS DE INFORMAÇÃO,Bacharelado,SÃO JOÃO EVANGELISTA,Turno Manhã e Tarde,Superior,3,1,0,1,Notas do Enem
86,SISTEMAS DE INFORMAÇÃO,Bacharelado,SÃO JOÃO EVANGELISTA,Turno Manhã e Tarde,Superior,0,0,0,0,Redação Online


In [81]:
df_tcc2024 = process_file_for_tec("dados/2024-1_GestaoResultado_ResumoInscricoes_TEC.xlsx")
df_tcc2025 = process_file_for_tec("dados/2025-1_GestaoResultado_ResumoInscricoes_TEC.xlsx")

In [83]:
df_tcc2024[df_tcc2024['Campus']=="PONTE NOVA"]


,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,Modalidade,FormaIngresso
4,ADMINISTRAÇÃO,Curso Técnico Integrado,PONTE NOVA,Turno Integral,Nível Fundamental,339,282,28,310,Curso Técnico Integrado,Processo Seletivo
30,INFORMÁTICA,Curso Técnico Integrado,PONTE NOVA,Turno Integral,Nível Fundamental,291,241,26,267,Curso Técnico Integrado,Processo Seletivo


In [85]:
df_tcc2025[df_tcc2025['Campus']=="PONTE NOVA"]

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,Modalidade,FormaIngresso
37,ADMINISTRAÇÃO,Curso Técnico Integrado,PONTE NOVA,Turnos Manhã e Tarde,médio,58,24,0,24,Curso Técnico Integrado,Processo Seletivo
38,INFORMÁTICA,Curso Técnico Integrado,PONTE NOVA,Turnos Manhã e Tarde,médio,73,24,0,24,Curso Técnico Integrado,Processo Seletivo


In [90]:
# Tecnicos
diff(process_file_for_tec("dados/2024-1_GestaoResultado_ResumoInscricoes_TEC.xlsx"),
process_file_for_tec("dados/2025-1_GestaoResultado_ResumoInscricoes_TEC.xlsx"), tipo="Campus")


,Campus,Inscritos,Pagos,Deferidas,Homologadas
2,BETIM,-1963,-1629,-178,-1807
11,OURO PRETO,-1344,-1022,-136,-1158
16,SANTA LUZIA,-1039,-696,-145,-841
15,SABARÁ,-1024,-842,-116,-958
14,RIBEIRÃO DAS NEVES,-998,-761,-156,-917
17,SÃO JOÃO EVANGELISTA,-922,-699,-147,-846
7,IBIRITÉ,-849,-721,-91,-812
6,GOVERNADOR VALADARES,-811,-568,-124,-692
3,CONGONHAS,-668,-488,-110,-598
8,IPATINGA,-654,-536,-56,-592


In [101]:
# Superiores
diff(process_file_for_grad("dados/2024-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx"),
process_file_for_grad("dados/2025-1_GestaoResultado_ResumoInscricoes_GRAD.xlsx"), tipo=["Campus"])


,Campus,Inscritos,Pagos,Deferidas,Homologadas
15,SANTA LUZIA,-406.0,-178.0,-112.0,-290.0
1,BAMBUÍ,-363.0,-225.0,-82.0,-307.0
9,OURO BRANCO,-317.0,-189.0,-86.0,-275.0
14,SABARÁ,-295.0,-143.0,-66.0,-209.0
2,BETIM,-292.0,-151.0,-64.0,-215.0
16,SÃO JOÃO EVANGELISTA,-250.0,-127.0,-61.0,-188.0
5,GOVERNADOR VALADARES,-229.0,-100.0,-48.0,-148.0
3,CONGONHAS,-173.0,-87.0,-46.0,-133.0
0,ARCOS,-148.0,-119.0,-2.0,-121.0
13,RIBEIRÃO DAS NEVES,-135.0,-44.0,-32.0,-76.0


In [108]:
ano = "2024-1"

dfMed = process_file_for_tec(f'dados/{ano}_GestaoResultado_ResumoInscricoes_TEC.xlsx')
dfGra = process_file_for_grad(f'dados/{ano}_GestaoResultado_ResumoInscricoes_GRAD.xlsx')



In [109]:
dfMed.head()

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,FormaIngresso
0,ADMINISTRAÇÃO,Curso Técnico Integrado,FORMIGA,Turno Integral,Nível Fundamental,84,54,14,68,Processo Seletivo
1,ADMINISTRAÇÃO,Curso Técnico Integrado,BAMBUÍ,Turno Integral,Nível Fundamental,66,47,4,51,Processo Seletivo
2,ADMINISTRAÇÃO,Curso Técnico Integrado,OURO BRANCO,Turno Integral,Nível Fundamental,221,171,25,196,Processo Seletivo
3,ADMINISTRAÇÃO,Curso Técnico Integrado,OURO PRETO,Turno Integral,Nível Fundamental,273,223,22,245,Processo Seletivo
4,ADMINISTRAÇÃO,Curso Técnico Integrado,PONTE NOVA,Turno Integral,Nível Fundamental,339,282,28,310,Processo Seletivo


In [110]:
dfGra.head()

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,FormaIngresso
0,ADMINISTRAÇÃO,Superior,BAMBUÍ,Turno Noturno,Nível Médio,44,24,4,28,Aproveitamento de notas obtidas no ENEM
1,ADMINISTRAÇÃO,Superior,FORMIGA,Turno Noturno,Nível Médio,68,40,4,44,Aproveitamento de notas obtidas no ENEM
2,ADMINISTRAÇÃO,Superior,OURO BRANCO,Turno Noturno,Nível Médio,133,74,26,100,Aproveitamento de notas obtidas no ENEM
3,ADMINISTRAÇÃO,Superior,RIBEIRÃO DAS NEVES,Turno Manhã,Nível Médio,72,22,15,37,Aproveitamento de notas obtidas no ENEM
4,ADMINISTRAÇÃO,Superior,SABARÁ,Turno Noturno,Nível Médio,88,27,21,48,Aproveitamento de notas obtidas no ENEM


In [113]:
df = pd.concat([dfGra, dfMed])
df

,Curso,Modalidade,Campus,Turno,Nivel,Inscritos,Pagos,Deferidas,Homologadas,FormaIngresso
0,ADMINISTRAÇÃO,Superior,BAMBUÍ,Turno Noturno,Nível Médio,44,24,4,28,Aproveitamento de notas obtidas no ENEM
1,ADMINISTRAÇÃO,Superior,FORMIGA,Turno Noturno,Nível Médio,68,40,4,44,Aproveitamento de notas obtidas no ENEM
2,ADMINISTRAÇÃO,Superior,OURO BRANCO,Turno Noturno,Nível Médio,133,74,26,100,Aproveitamento de notas obtidas no ENEM
3,ADMINISTRAÇÃO,Superior,RIBEIRÃO DAS NEVES,Turno Manhã,Nível Médio,72,22,15,37,Aproveitamento de notas obtidas no ENEM
4,ADMINISTRAÇÃO,Superior,SABARÁ,Turno Noturno,Nível Médio,88,27,21,48,Aproveitamento de notas obtidas no ENEM
...,...,...,...,...,...,...,...,...,...,...
66,SEGURANÇA DO TRABALHO,-1,GOVERNADOR VALADARES,Turno Noturno,Nível Médio,1,1,0,1,Processo Seletivo
67,SEGURANÇA DO TRABALHO,Curso Técnico Subsequente,GOVERNADOR VALADARES,Turno Noturno,Nível Médio,135,29,19,48,Processo Seletivo
68,SEGURANÇA DO TRABALHO,-1,OURO PRETO,Turno Noturno,Nível Médio,1,1,0,1,Processo Seletivo
69,SEGURANÇA DO TRABALHO,Curso Técnico Subsequente,OURO PRETO,Turno Noturno,Nível Médio,194,70,23,93,Processo Seletivo


In [114]:
ano1 = "2024-1"
ano2 = "2025-1"

dffTC = diff(process_file_for_tec(f'dados/{ano1}_GestaoResultado_ResumoInscricoes_TEC.xlsx'), 
            process_file_for_tec(f'dados/{ano2}_GestaoResultado_ResumoInscricoes_TEC.xlsx'), 
            "Campus")

dffGC = diff(process_file_for_grad(f'dados/{ano1}_GestaoResultado_ResumoInscricoes_GRAD.xlsx'), 
            process_file_for_grad(f'dados/{ano2}_GestaoResultado_ResumoInscricoes_GRAD.xlsx'), 
            "Campus")

In [119]:
dffTC

,Campus,Inscritos,Pagos,Deferidas,Homologadas
2,BETIM,-1963,-1629,-178,-1807
11,OURO PRETO,-1344,-1022,-136,-1158
16,SANTA LUZIA,-1039,-696,-145,-841
15,SABARÁ,-1024,-842,-116,-958
14,RIBEIRÃO DAS NEVES,-998,-761,-156,-917
17,SÃO JOÃO EVANGELISTA,-922,-699,-147,-846
7,IBIRITÉ,-849,-721,-91,-812
6,GOVERNADOR VALADARES,-811,-568,-124,-692
3,CONGONHAS,-668,-488,-110,-598
8,IPATINGA,-654,-536,-56,-592


In [120]:
fig7 = px.bar(dffTC, x="Campus", y=["Inscritos","Homologadas"], color="Campus",template={}, text_auto='.2s')
fig7.show()

In [118]:
px.bar(dffGC, x="Campus", y=["Homologadas"], color="Campus", text_auto='.2s').show()